# Installing Modules

In [ ]:
!pip install wandb -qU
!pip install pytorch_lightning


# Importing Modules


In [2]:
import wandb
import torch 
import pytorch_lightning as pl
import torch.nn as nn
from torch.nn  import functional
from pytorch_lightning.loggers import WandbLogger
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data
from torchvision import models
import numpy as np
import random

# Downloading Unzip data

In [ ]:
!curl -SL https://storage.googleapis.com/wandb_datasets/nature_12K.zip > nature_12K.zip
!unzip nature_12K.zip


# Connecting Wandb


In [4]:
wandb.login(key = "8d6c17aa48af2229c26cbc16513ef266358c0b96")
wandb.init(project="Assignment-02-resnet",entity = "saisreeram")
wandb.run.name = "resnet18"

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs22m076 (saisreeram). Use `wandb login --relogin` to force relogin


# Data Loading

In [5]:
baseDir = "inaturalist_12K"
trainDir = baseDir+"/train/"
testDir = baseDir+"/val/"
outputclasses=["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]
input_size = 256
batch_size = 16 


transform = transforms.Compose([
            transforms.Resize((input_size,input_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
        ])

train_dataset = datasets.ImageFolder(root=baseDir+'/train',transform=transform)
test_dataset = datasets.ImageFolder(root=baseDir+'/val',transform=transform)
 
trainSize = int(0.8 * len(train_dataset))
valSize = len(train_dataset) - trainSize

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [trainSize, valSize])

train_dataset = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           )

val_dataset = torch.utils.data.DataLoader(val_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           )

test_dataset = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           )


# Pre - Trained

In [6]:
class Resnet(pl.LightningModule):
  def __init__(self):
    super(Resnet,self).__init__()

    self.train_step_acc = []
    self.train_step_loss = []
    self.val_step_acc = []
    self.val_step_loss = []

    self.resnet = models.resnet18(pretrained=True)
    num_features = self.resnet.fc.in_features
    self.resnet.fc = torch.nn.Linear(num_features, 10) # replace 10 with number of output classes

  def forward(self,x):
      return self.resnet(x)

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(),lr = 0.0001)

  def training_step(self,batch):
    trainX,trainY = batch
    output = self(trainX)

    loss = functional.cross_entropy(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()

    self.train_step_acc.append(acc)
    self.train_step_loss.append(loss)

    self.log('train_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('train_acc', acc,on_epoch = True,on_step = False,prog_bar=True)

    return loss

  def on_train_epoch_end(self):
    
    train_acc =  torch.stack(self.train_step_acc).mean()
    train_loss =  torch.stack(self.train_step_loss).mean()

    val_acc =  torch.stack(self.val_step_acc).mean()
    val_loss =  torch.stack(self.val_step_loss).mean()

    wandb.log({"train_loss":train_loss.item(),"train_acc":train_acc.item(),"val_loss":val_loss.item(),"val_acc":val_acc.item()})
    self.train_step_acc.clear() 
    self.train_step_loss.clear() 

    self.val_step_acc.clear() 
    self.val_step_loss.clear() 

  def validation_step(self, batch,batch_idx):
    trainX,trainY = batch
    output = self(trainX)

    loss = functional.cross_entropy(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()

    self.val_step_acc.append(acc)
    self.val_step_loss.append(loss)

    self.log('val_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('val_acc', acc,on_epoch = True,on_step = False,prog_bar=True)

    return loss

  def test_step(self, batch,batch_idx):
    trainX,trainY = batch
    output = self(trainX)

    loss = functional.cross_entropy(output,trainY)
    acc = (output.argmax(dim = 1) == trainY).float().mean()

    self.log('test_loss', loss,on_epoch = True,on_step = False,prog_bar=True)
    self.log('test_acc', acc,on_epoch = True,on_step = False,prog_bar=True)

    return loss



In [7]:
Resnet1 = Resnet()
trainer = pl.Trainer(max_epochs=5)
trainer.fit(Resnet1,train_dataset,val_dataset)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.7MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.util

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [8]:
trainer.test(Resnet1, test_dataset)


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7455000281333923     │
│         test_loss         │    0.9259812235832214     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.9259812235832214, 'test_acc': 0.7455000281333923}]